In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
import sqlalchemy
from sqlalchemy import create_engine
import pymysql
import mysql.connector
from config import mongoConn, remote_db_endpoint, remote_db_port, remote_db_name, remote_db_user, remote_db_pwd

In [7]:
#MySQLdb insert connection
conn = pymysql.connect(host=f'{remote_db_endpoint}',
                             user=remote_db_user,
                             password=remote_db_pwd,
                             db=remote_db_name)
mycursor = conn.cursor()

In [23]:
#Function insert servivices into table
def insert_service(VodDict ):
    sql = """
    INSERT INTO 
        streamingservices(Service_Name, Service_Type, Service_Img, Service_URL, Popular)
    VALUES 
        (%s, %s, %s, %s, %s);
        """ 
    if VodDict['Service_Name'] == '':
        return False
    if VodDict['Service_Name'] == 'Free Services Bundle':
        return False    
     
    POP = 0
    if VodDict['Service_Name'] in PopularVod:
        POP = 10 
    val = (VodDict['Service_Name'], VodDict['Service_Type'], VodDict['Service_Img'], VodDict['Service_URL'], POP)
    mycursor.execute(sql, val)
    conn.commit()
    return True


In [25]:
#Service name sanitizer, create url for tables
def getURL(name):
    name = name.replace(' ', '')
    name = name.replace('+', '')
    name = name + '.com'
    return name


In [26]:
#service scraper:
#url to scrape
url = 'https://reelgood.com/services'
# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
#target scrape
services = soup.select('ul.e2kiuu0 a')
vodCount = 0
PopularVod= []
for service in services:
    #veriables for scrape
    img = service.select('img')[0]['src']
    name = service.select('h3')[0].text
    url = getURL(name)
    VodDict = {'Service_Name':name,  'Service_Img':img, 'Service_URL':url}
    #loop selects services calls function to populate table
    if name == '':
        vodCount += 1
    if vodCount == 0:
        PopularVod.append(name)
    if vodCount == 4:
        VodDict['Service_Type']='AVOD'
        insert_service(VodDict)
    if vodCount == 8:
        VodDict['Service_Type']='SVOD'
        insert_service(VodDict)
    if vodCount == 12:
        VodDict['Service_Type']='TVOD'
        insert_service(VodDict)
    
 

In [ ]:
#Insert for user_profile table
def insert_user(profile_dict):
    sql = """
    INSERT INTO 
        user_profile(User_Name, Age, Gender, Watch_Frequency, Zip_Code)
    VALUES 
        (%s, %s, %s, %s, %s);
        """ 
    val = (profile_dict['User_Name'], profile_dict['Age'], profile_dict['Gender'], profile_dict['Watch_Frequency'], profile_dict['Zip_Code'])
    mycursor.execute(sql, val)
    conn.commit()

#Delete records
sql = """
    DELETE FROM 
        streamingservices;
    """
mycursor.execute(sql) 
conn.commit()

#connect to MySQLdb
pymysql.install_as_MySQLdb()
connstr = f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}"
engine = create_engine(connstr) 